#Below is our Web Scraper in action. The scraper goes through over 100,000 pages of TripAdvisor to scrape the reviews posted on the website for each National Park.
Reference: https://github.com/susanli2016/NLP-with-Python/blob/master/Web%20scraping%20Hilton%20Hawaiian%20Village%20TripAdvisor%20Reviews.py

In [0]:
### TOTAL CODE TOGETHER
## Amended from Susan Li - https://github.com/susanli2016/NLP-with-Python/blob/master/Web%20scraping%20Hilton%20Hawaiian%20Village%20TripAdvisor%20Reviews.py
## Code below enhanced/forked by Hien Dang and Charlie Leifheit


import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io
import re
import pandas as pd

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

# get URL sessions to parse with BeautifulSoup        
def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
      
#ensure session access succesful, then read page with BeautifulSoup      
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
#find all items on the page in the selected language = English
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items
  

#create a variable to log the number of reviews identified per park
total_reviews_per_park = []

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return
    # get the number of reviews in the soup, then conver to integer
    num_reviews = soup.find('span', class_='reviewCount').text # get text
    num_reviews = re.sub("[^0-9]", "",num_reviews)
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    print('[parse] num_reviews ALL:', num_reviews)
    
    #set URL template to parse through each page of reviews - which increase by 10 every page
    url_template = url.replace('Reviews-', 'Reviews-or{}-')
    print('[parse] url_template:', url_template)
    
    #get the park name or Title of the page
    park_name = soup.find('h1', id='HEADING').text
    if [park_name,num_reviews] not in total_reviews_per_park:
      total_reviews_per_park.append([park_name,num_reviews])
    
    #set list for every review items which will be pulled below
    items = []

    offset = 0
    
    #while the offset variable is less than the total number of reviews, keep pulling pages
    #TripAdvisor increments reviews 10 per page
    while offset <= num_reviews:

        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 9:
            break

        offset += 10

    return items

  #all data is found using the review ids on each page - parsing from BEautifulSoup
def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        return reviews_ids
    
def get_more(session, reviews_ids):

  ##The widget overlay allows us to easily find the information we need - "hotel reviews" and "attraction reviews" function the same

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

#loop review reviews grabbed above and pull pertinent information
def parse_reviews(session, url):
    '''Get all reviews from one page'''

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return
    # Find the park name in the Heading of the HTML soup
    park_name = soup.find('h1', id='HEADING').text
    print(park_name)
    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return


    items = []

    #iterate through all reviews pulling the necessary data from each
    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'
        #get the number of helpful votes listed per review
        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'

        #get the location the user identifies on their post (this is their "home" location)
        user_loc = review.find('div', class_ = 'userLoc')

        if user_loc is None:
            user_loc = ''
        elif len(user_loc) > 0:
            user_loc = user_loc.text
        else:
            user_loc = ''

        ## having trouble finding the user ID - "data-screenName="AKJazz"
        # We ended up not using this portion of the code, but keeping it here for future use
        #user_id = review.select_one("ta.trackEventOnPage")
        #print(user_id)
        #if user_id:
        #    user_id = user_id.text
        #else:
        #    user_id = ''  
        #print (user_id)
        
        #get the user rating - on a 5 bubble scale
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        bubble_rating = bubble_rating[0]
        
        #post all items to our "items" list
        item = {
            'national_park': park_name,
            'review_body': review.find('p', class_='partial_entry').text,
            'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'review_title': review.find('span', class_='noQuotes').text,
            'helpful_vote': helpful_vote,
            'user_location': user_loc,
            'user_rating': bubble_rating


        }

        items.append(item)

    return items

#write the results into a CSV
def write_in_csv(items, filename='results.csv',
                  headers=['national_park', 'review_body','review_date', 'review_title',
                           'helpful_vote', 'user_location', 'user_rating'],
                  mode='w'):

    #print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'


#list all starting urls we'll parse data from for Trip Advisor
#It was easier to past the static URLS to the Jupyter, rather than pull in the Google Drive doc.

start_urls = [
'https://www.tripadvisor.com/Attraction_Review-g60709-d12785836-Reviews-Acadia_National_Park-Bar_Harbor_Mount_Desert_Island_Maine.html',
'https://www.tripadvisor.com/Attraction_Review-g60672-d6776101-Reviews-National_Park_of_American_Samoa-Pago_Pago_Tutuila.html',
'https://www.tripadvisor.com/Attraction_Review-g60724-d8535374-Reviews-Arches_National_Park-Moab_Utah.html',
'https://www.tripadvisor.com/Attraction_Review-g60729-d10212205-Reviews-Badlands_National_Park-Interior_South_Dakota.html',
'https://www.tripadvisor.com/Attraction_Review-g60733-d144706-Reviews-Chisos_Mountains-Big_Bend_National_Park_Texas.html',
'https://www.tripadvisor.com/Attraction_Review-g60739-d1858774-Reviews-Biscayne_National_Park-Homestead_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g143014-d145009-Reviews-Black_Canyon_Of_The_Gunnison_National_Park-Black_Canyon_Of_The_Gunnison_National_P.html',
'https://www.tripadvisor.com/Attraction_Review-g143015-d1925256-Reviews-Bryce_Canyon_National_Park-Bryce_Canyon_National_Park_Utah.html',
'https://www.tripadvisor.com/Attraction_Review-g60724-d8837806-Reviews-Canyonlands_National_Park-Moab_Utah.html',
'https://www.tripadvisor.com/Attraction_Review-g143017-d212117-Reviews-Capitol_Reef_National_Park-Capitol_Reef_National_Park_Utah.html',
'https://www.tripadvisor.com/Attraction_Review-g143018-d144825-Reviews-Big_Room-Carlsbad_Caverns_National_Park_New_Mexico.html',
'https://www.tripadvisor.com/Attraction_Review-g143019-d141863-Reviews-Santa_Cruz_Island-Channel_Islands_National_Park_California.html',
'https://www.tripadvisor.com/Attraction_Review-g60777-d143108-Reviews-Congaree_National_Park-Hopkins_South_Carolina.html',
'https://www.tripadvisor.com/Attraction_Review-g143020-d144747-Reviews-Crater_Lake_National_Park-Crater_Lake_National_Park_Oregon.html',
'https://www.tripadvisor.com/Attraction_Review-g60784-d261325-Reviews-Cuyahoga_Valley_National_Park-Brecksville_Ohio.html',
'https://www.tripadvisor.com/Attraction_Review-g143021-d531585-Reviews-Dante_s_View-Death_Valley_National_Park_Inyo_County_California.html',
'https://www.tripadvisor.com/Attraction_Review-g143022-d145106-Reviews-Denali-Denali_National_Park_and_Preserve_Alaska.html',
'https://www.tripadvisor.com/Attraction_Review-g34345-d3417409-Reviews-Dry_Tortugas_National_Park-Key_West_Florida_Keys_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34225-d6851227-Reviews-Everglades_National_Park-Florida_City_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g143025-d208203-Reviews-Koyukuk_River-Gates_Of_The_Arctic_National_Park_and_Preserve_Alaska.html',
'https://www.tripadvisor.com/Attraction_Review-g44881-d104318-Reviews-The_Gateway_Arch-Saint_Louis_Missouri.html',
'https://www.tripadvisor.com/Attraction_Review-g143026-d146551-Reviews-Going_to_the_Sun_Road-Glacier_National_Park_Montana.html',
'https://www.tripadvisor.com/Attraction_Review-g143027-d537964-Reviews-Glacier_Bay_National_Park_Preserve-Glacier_Bay_National_Park_and_Preserve_Alaska.html',
'https://www.tripadvisor.com/Attraction_Review-g143028-d109440-Reviews-Grand_Canyon_South_Rim-Grand_Canyon_National_Park_Arizona.html',
'https://www.tripadvisor.com/Attraction_Review-g143029-d145270-Reviews-Grand_Teton-Grand_Teton_National_Park_Wyoming.html',
'https://www.tripadvisor.com/Attraction_Review-g60840-d1584499-Reviews-Great_Basin_National_Park-Baker_Nevada.html',
'https://www.tripadvisor.com/Attraction_Review-g60841-d9800243-Reviews-Great_Sand_Dunes_National_Park-Mosca_Colorado.html',
'https://www.tripadvisor.com/Attraction_Review-g60842-d12833528-Reviews-Great_Smoky_Mountains_National_Park-Gatlinburg_Tennessee.html',
'https://www.tripadvisor.com/Attraction_Review-g143032-d144713-Reviews-Guadalupe_Peak-Guadalupe_Mountains_National_Park_Texas.html',
'https://www.tripadvisor.com/Attraction_Review-g143033-d103255-Reviews-Haleakala_Crater-Haleakala_National_Park_Maui_Hawaii.html',
'https://www.tripadvisor.com/Attraction_Review-g143034-d1603722-Reviews-Hawaii_Volcanoes_National_Park-Hawaii_Volcanoes_National_Park_Island_of_Hawaii_Ha.html',
'https://www.tripadvisor.com/Attraction_Review-g60856-d3224470-Reviews-Hot_Springs_National_Park-Hot_Springs_Arkansas.html',
'https://www.tripadvisor.com/Attraction_Review-g37006-d259610-Reviews-Indiana_Dunes_State_Park-Chesterton_Indiana.html',
'https://www.tripadvisor.com/Attraction_Review-g143036-d145184-Reviews-Rock_Harbor-Isle_Royale_National_Park_Michigan.html',
'https://www.tripadvisor.com/Attraction_Review-g32544-d13955290-Reviews-Joshua_Tree_National_Park-Joshua_Tree_California.html',
'https://www.tripadvisor.com/Attraction_Review-g143038-d145317-Reviews-Brooks_Falls-Katmai_National_Park_and_Preserve_Alaska.html',
'https://www.tripadvisor.com/Attraction_Review-g60873-d3377126-Reviews-Kenai_Fjords_National_Park-Seward_Alaska.html',
'https://www.tripadvisor.com/Attraction_Review-g143050-d145249-Reviews-Kings_Canyon-Sequoia_and_Kings_Canyon_National_Park_California.html',
'https://www.tripadvisor.com/Attraction_Review-g143040-d12268414-Reviews-Kobuk_Valley_National_Park-Kobuk_Valley_National_Park_Alaska.html',
'https://www.tripadvisor.com/Attraction_Review-g143041-d12268413-Reviews-Lake_Clark_National_Park_and_Preserve-Lake_Clark_National_Park_and_Preserve_Alas.html',
'https://www.tripadvisor.com/Attraction_Review-g143042-d146513-Reviews-Mount_Lassen-Lassen_Volcanic_National_Park_California.html',
'https://www.tripadvisor.com/Attraction_Review-g143043-d146944-Reviews-Mammoth_Cave-Mammoth_Cave_National_Park_Kentucky.html',
'https://www.tripadvisor.com/Attraction_Review-g60900-d125453-Reviews-Cliff_Palace-Mesa_Verde_National_Park_Colorado.html',
'https://www.tripadvisor.com/Attraction_Review-g143044-d145643-Reviews-Mount_Rainier-Mount_Rainier_National_Park_Washington.html',
'https://www.tripadvisor.com/Attraction_Review-g143046-d12409123-Reviews-North_Cascades_National_Park-North_Cascades_National_Park_Washington.html',
'https://www.tripadvisor.com/Attraction_Review-g143047-d145462-Reviews-Olympic_National_Forest-Olympic_National_Park_Washington.html',
'https://www.tripadvisor.com/Attraction_Review-g60932-d102672-Reviews-Painted_Desert-Petrified_Forest_National_Park_Arizona.html',
'https://www.tripadvisor.com/Attraction_Review-g60935-d143240-Reviews-Pinnacles_National_Park-Paicines_California.html',
'https://www.tripadvisor.com/Attraction_Review-g2193168-d210214-Reviews-Redwood_National_Park-Redwood_National_Park_California.html',
'https://www.tripadvisor.com/Attraction_Review-g28927-d146464-Reviews-Rocky_Mountains-Colorado.html',
'https://www.tripadvisor.com/Attraction_Review-g60950-d12704675-Reviews-Saguaro_National_Park-Tucson_Arizona.html',
'https://www.tripadvisor.com/Attraction_Review-g33230-d12902316-Reviews-Sequoia_National_Park-Visalia_California.html',
'https://www.tripadvisor.com/Attraction_Review-g28967-d477530-Reviews-Shenandoah_Valley-Virginia.html',
'https://www.tripadvisor.com/Attraction_Review-g60973-d1911641-Reviews-Theodore_Roosevelt_National_Park-Medora_North_Dakota.html',
'https://www.tripadvisor.com/Attraction_Review-g147411-d208083-Reviews-National_Park_Underwater_Trail-Virgin_Islands_National_Park_St_John_U_S_Virgin_Isl.html',
'https://www.tripadvisor.com/Attraction_Review-g43447-d12659479-Reviews-Voyageurs_National_Park-Ray_Minnesota.html',
'https://www.tripadvisor.com/Attraction_Review-g143055-d10162780-Reviews-Wind_Cave_National_Park-Wind_Cave_National_Park_South_Dakota.html',
'https://www.tripadvisor.com/Attraction_Review-g143056-d4573297-Reviews-Wrangell_St_Elias_National_Park_Preserve_Visitor_Center-Wrangell_St_Elias_Nationa.html',
'https://www.tripadvisor.com/Attraction_Review-g60999-d12235309-Reviews-Yellowstone_National_Park-Yellowstone_National_Park_Wyoming.html',
'https://www.tripadvisor.com/Attraction_Review-g61000-d489919-Reviews-Yosemite_Valley-Yosemite_National_Park_California.html',
'https://www.tripadvisor.com/Attraction_Review-g143057-d103336-Reviews-Zion_s_Main_Canyon-Zion_National_Park_Utah.html'
]

lang = 'en'

#set headers for the csv
headers = ['national_park', 'review_body','review_date', 'review_title',
                           'helpful_vote', 'user_location', 'user_rating']

#post results into csv for each park - we'll concatenate later
for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)
    if not items:
        print('No reviews')
    else:
        #write in CSV
        reviews_df = pd.DataFrame(items)
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')

''' 
The resulting CSV files will store in the "temp" memory of Google Collaboratory.

This can be found in the "Files" tab on the left hand side of yoru screen

When running on a local machine, the files will store to your local.

For this project we uploaded the files to a Google Drive and GitHub

'''


[parse] url: https://www.tripadvisor.com/Attraction_Review-g60709-d12785836-Reviews-Acadia_National_Park-Bar_Harbor_Mount_Desert_Island_Maine.html?filterLang=en
[parse] num_reviews ALL: 1621
[parse] url_template: https://www.tripadvisor.com/Attraction_Review-g60709-d12785836-Reviews-or{}-Acadia_National_Park-Bar_Harbor_Mount_Desert_Island_Maine.html?filterLang=en
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acadia National Park
Acad

### **REVIEWS DATAFRAME OUTPUT**

In [0]:
##checking that the total_reviews_per_park matches our total CSV ouput

total_reviews_per_park
total_reviews= 0
for review in total_reviews_per_park:
  total_reviews += int(review[1])


NameError: ignored

##Cell 7
### Loading a new session in Google Colab will generally require you to load the data again

### The outputs csv for each park has been saved to a mounted Google Drive using an email created for this project.  

These files have since been uploaded to a GitHub repo.  The following ZIP file must be downloaded from Github:

https://github.gatech.edu/DVA-team97/TripAdvisor-Web-Scraper/blob/master/TripAdvisor_National_Park_Reviews_CSV_Files.zip?raw=true

After downloading - the ZIP file must be uploaded to the Google Colaboratory Session using the Files>Upload function on the LEFT



<<<<<<<<<<<<<<<<<<<<<<






In [40]:
#load all review data from csv's in mounted GoogleDrive
import os
from os import listdir
from os.path import isfile
import glob
import pandas as pd
import zipfile

#set path to Google Colab

mypath = ('/content/')


import zipfile

#set variable as path

with zipfile.ZipFile('/content/TripAdvisor_National_Park_Reviews_CSV_Files.zip',"r") as zip_ref:
    zip_ref.extractall(mypath)




#set all file names to variable

files = glob.glob('/content/TripAdvisor_National_Park_Reviews_CSV_Files/*.csv')
print("The number of parks in the GitHub is:" + str(len(files)))

#for each file name - replace the folder path and keep only the file name in a list
csv_names = []
for file in files:
    csv_n = file.replace(mypath, '')

    csv_names.append(csv_n)

#import all files in pandas data frame

merged_reviews = pd.DataFrame()

df_from_each_file = (pd.read_csv(csv) for csv in csv_names)
merged_reviews   = pd.concat(df_from_each_file, ignore_index=True)

merged_reviews.tail()


The number of parks in the GitHub is:61


,national_park,review_body,review_date,review_title,helpful_vote,user_location,user_rating
115064,Going-to-the-Sun Road,This is probably one of the most beautiful dri...,"October 8, 2019",Beautiful Drive,21,"Midlothian, Virginia",5
115065,Going-to-the-Sun Road,"Yes, you have to make sure it’s open before yo...","October 7, 2019",Beautiful!!!!,157,NaN,5
115066,Going-to-the-Sun Road,Drove up all the way from Utah only to find th...,"October 7, 2019",Check it’s open before you go,327,"Salt Lake City, Utah",3
115067,Going-to-the-Sun Road,An unforgettable experience made much more enj...,"October 7, 2019",Park and Ride,222,"New Bloomfield, Pennsylvania",5
115068,Going-to-the-Sun Road,It was a long road to get to this road (from O...,"October 2, 2019",Gorgeous,3,"Omaha, Nebraska",5


In [38]:
##determine our base of words to ensure we have a "large" dataset
#set word count variable
word_count = 0
title_word_count = 0


#count the length of each word in each review to update word_count
for review in merged_reviews.review_body:
  word_count += len(review.split())

for review in merged_reviews.review_title:
  title_word_count += (len(review.split()))

title_word_count_weighted = title_word_count * 2

print ("Total number of words in the review and title (unweighted) is: " + str(word_count + title_word_count))
print ('Total number of words in the review and titles (weighted 2X for title words) is: ' + str(word_count + title_word_count_weighted))

Total number of words in the review and title (unweighted) is: 10816566
Total number of words in the review and titles (weighted 2X for title words) is: 11291885


### We will now begin cleaning the dataframe using Pandas

### This will allow our Feature Enginnering and Modeling phase to move a bit smoother

In [0]:
#first, copy df into new variable to retain original merged_reviews
df_cleaning = merged_reviews.copy()

#determine column types for formatting
df_cleaning.dtypes

#convert review_date to datetime format for parsing
df_cleaning.review_date = pd.to_datetime(df_cleaning.review_date, infer_datetime_format = True)

#confirm formatted
df_cleaning.head()

#create new column for month, day, and year

#month
df_cleaning['review_month'] = df_cleaning.review_date.dt.month
df_cleaning.head()

#day
df_cleaning['review_day'] = df_cleaning.review_date.dt.day
df_cleaning.head()

#year
df_cleaning['review_year'] = df_cleaning.review_date.dt.year
df_cleaning.head()

,national_park,review_body,review_date,review_title,helpful_vote,user_location,user_rating,review_month,review_day,review_year
0,Cliff Palace,Mesa Verde's Cliff Palace House is not to be m...,2019-10-18,A Palce Indeed,49,"Kent, Washington",5,10,18,2019
1,Cliff Palace,Informative tour! The ranger did a good job of...,2019-10-18,Throughly enjoyed our group tour with ranger,101,"Brookhaven, Georgia",4,10,18,2019
2,Cliff Palace,I visited wonders of the World Machu Piccu and...,2019-10-16,Amazing,18,"Delden, The Netherlands",5,10,16,2019
3,Cliff Palace,You could not walk all they way down because i...,2019-10-13,See the whole park,0,"Central Arizona, Arizona",5,10,13,2019
4,Cliff Palace,"Exceptional, breathtaking and amazing! You wil...",2019-10-10,MUST SEE IF YOU ARE IN THE AREA1,43,"Springfield, Missouri",5,10,10,2019


### We're going to visualize the table just to ensure we have correct data


In [0]:
df_cleaning.sort_values(by =['user_rating'], ascending = True)
df_cleaning.sort_values(by =['helpful_vote'], ascending = False).head(15)

,national_park,review_body,review_date,review_title,helpful_vote,user_location,user_rating,review_month,review_day,review_year
113969,Yellowstone National Park,There are more than 300 famous geysers include...,2019-09-11,A geyser wonderland,83946,"Yekaterinburg, Russia",5,9,11,2019
59566,Grand Canyon South Rim,Amazing rock formations over a 277 mile length...,2019-09-09,Phenomenally spectacular,83929,"Yekaterinburg, Russia",5,9,9,2019
8709,Arches National Park,"WOW . . . This park is home to over 2,000 amaz...",2018-10-22,Awe-inspiring sandstone arches,83927,"Yekaterinburg, Russia",5,10,22,2018
22657,Bryce Canyon National Park,"At 145 square kilometres, this is a large park...",2018-10-22,Spectacular photo ops at sunrise and sunset,83927,"Yekaterinburg, Russia",5,10,22,2018
18399,Capitol Reef National Park,We started by watching the movie in the visito...,2018-10-23,"Stunningly beautiful scenery, rock formations,...",83927,"Yekaterinburg, Russia",5,10,23,2018
35034,Canyonlands National Park,"Unlike the first two parks I went to in Utah, ...",2018-10-22,Awesome rock formations & remarkable landscapes,83927,"Yekaterinburg, Russia",5,10,22,2018
50602,Zion's Main Canyon,The main canyon area in Zion is situated betwe...,2017-06-27,"Remarkable beauty in all directions, great hik...",18552,"Hong Kong, China",5,6,27,2017
109446,Yosemite Valley,The Yosemite Valley is the heart of Yosemite N...,2019-05-20,The Heart of Yosemite National Park,18552,"Hong Kong, China",5,5,20,2019
42570,Sequoia National Park,We recently visited several national parks in ...,2019-05-22,Land of the giant sequoias,18552,"Hong Kong, China",5,5,22,2019
113806,Yosemite Valley,We only spent a half day in the Yosemite Valle...,2011-09-22,"Excellent, well worth 2-3 days minimum",18552,"Hong Kong, China",4,9,22,2011


In [0]:

#create a copy of the dataframe for cleaning
df_working = df_cleaning.copy()
#Sort the DF by national park name ascending
df_working = df_working.sort_values(by =['national_park'], ascending = True)

#reset the index
df_working.reset_index(drop = True, inplace = True)
df_working.head()

KeyError: ignored

#Tabula data extraction for Visitor Spending Effect on each Nation Park

##First cell must be run to change working directory to the spending_files

##You must also install tabula when running a new session



In [0]:
import os
from os import listdir
from os.path import isfile
import glob
import pandas as pd


#change the path to a new spending folder:

os.chdir('/content/drive/My Drive/spending_files')

In [0]:
!pip install -q tabula-py

     |████████████████████████████████| 10.4MB 3.1MB/s 


In [0]:
import tabula
import pandas as pd
tabula.environment_info()

Python version:
    3.6.8 (default, Oct  7 2019, 12:59:55) 
[GCC 8.3.0]
Java version:
    openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
tabula-py version: 1.4.1
platform: Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
uname:
    uname_result(system='Linux', node='4af85c8d5387', release='4.14.137+', version='#1 SMP Thu Aug 8 02:47:02 PDT 2019', machine='x86_64', processor='x86_64')
linux_distribution: ('Ubuntu', '18.04', 'bionic')
mac_ver: ('', ('', '', ''), '')
    


In [0]:
# Import necessary libraries
import tabula
import pandas as pd

# Get 2012 dataset set from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path = "https://nthp-savingplaces.s3.amazonaws.com/2017/04/25/14/43/37/797/NPSVSE2012_final_nrss.pdf"
tabula.convert_into(pdf_path,"visitor_spent_2012.csv",output_format="csv",pages = [25,26,27,28,29,30,31,32,33,34],stream=True)

#Import the CSV created above into a dataframe
df_2012 = pd.read_csv('visitor_spent_2012.csv')

#create a new dataframe for the National Park Spending in 2012
spending_2012 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
#NOTE: rows 0 and 1 of the df_2012 contain the headers - which were grabbed by Tabula as two rows - so we ignore those
spending_2012['park_unit'] = df_2012.iloc[:,0][2:]
spending_2012['recreation_visits'] = df_2012.iloc[:,1][2:]
spending_2012['visitor_spending_$000s']=df_2012.iloc[:,2][2:]
spending_2012['year'] = 2012


#take a look at the top results of newly created table
spending_2012.head()



,year,park_unit,recreation_visits,visitor_spending_$000s
2,2012,Abraham Lincoln Birthplace NHP,"169,515","$9,121.8"
3,2012,Acadia NP,"2,431,052","$200,920.0"
4,2012,Adams NHP,"336,031","$18,082.2"
5,2012,African Burial Ground NM,"77,624","$4,177.0"
6,2012,Agate Fossil Beds NM*,"11,824",$798.9


In [0]:
# Get 2013 dataset set from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path_13 = "http://www.parkpartners.org/2013-National-Park-Visitor-Spending-Effects.html"
tabula.convert_into(pdf_path_13,"visitor_spent_2013.csv",output_format="csv",pages = [25,26,27,28,29,30,31,32,33,34],stream=True)

#Import the CSV created above into a dataframe
df_2013 = pd.read_csv('visitor_spent_2013.csv')

#create a new dataframe for the National Park Spending in 2013
spending_2013 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
spending_2013['park_unit'] = df_2013.iloc[:,0][6:]
spending_2013['recreation_visits'] = df_2013.iloc[:,1][6:]
spending_2013['visitor_spending_$000s']=df_2013.iloc[:,2][6:]
spending_2013['year'] = 2013

spending_2013.head()

,year,park_unit,recreation_visits,visitor_spending_$000s
6,2013,Abraham Lincoln Birthplace NHP,"163,928","$9,035.7"
7,2013,Acadia NP,"2,254,922","$190,953.9"
8,2013,Adams NHP,"285,500","$15,736.6"
9,2013,African Burial Ground NM,"93,234","$5,139.1"
10,2013,Agate Fossil Beds NM*,"10,503",$726.9


In [0]:
# Get 2014 dataset set from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path_14 = "http://npshistory.com/publications/social-science/visitor-spending/2014.pdf"
tabula.convert_into(pdf_path_14,"visitor_spent_2014.csv",output_format="csv",pages = [23,24,25,26,27,28,29,30,31,32],stream=True)

#Import the CSV created above into a dataframe
df_2014 = pd.read_csv('visitor_spent_2014.csv')

#create a new dataframe for the National Park Spending in 2013
spending_2014 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
spending_2014['park_unit'] = df_2013.iloc[:,0][6:]
spending_2014['recreation_visits'] = df_2013.iloc[:,1][6:]
spending_2014['visitor_spending_$000s']=df_2013.iloc[:,2][6:]
spending_2014['year'] = 2014

spending_2014.head()

,year,park_unit,recreation_visits,visitor_spending_$000s
6,2014,Abraham Lincoln Birthplace NHP,"163,928","$9,035.7"
7,2014,Acadia NP,"2,254,922","$190,953.9"
8,2014,Adams NHP,"285,500","$15,736.6"
9,2014,African Burial Ground NM,"93,234","$5,139.1"
10,2014,Agate Fossil Beds NM*,"10,503",$726.9


In [0]:
# Get 2015 dataset from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path_15 = "https://wolfwatcher.org/wp-content/uploads/2016/11/Visitor-Spending-Effects-NPS.pdf"
tabula.convert_into(pdf_path_15,"visitor_spent_2015.csv",output_format="csv",pages = [22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37],stream=True)

#Import the CSV created above into a dataframe
df_2015 = pd.read_csv('visitor_spent_2015.csv')

#create a new dataframe for the National Park Spending in 2015
spending_2015 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
spending_2015['park_unit'] = df_2015.iloc[:,0][8:]
spending_2015['recreation_visits'] = df_2015.iloc[:,1][8:]
spending_2015['visitor_spending_$000s']=df_2015.iloc[:,2][8:]
spending_2015['year'] = 2015
spending_2015.head()


,year,park_unit,recreation_visits,visitor_spending_$000s
8,2015,Abraham Lincoln Birthplace NHP,"148,605","$8,509.3"
9,2015,Acadia NP,"2,811,184","$247,910.8"
10,2015,Adams NHP,"183,632","$10,515.0"
11,2015,African Burial Ground NM,"56,348","$3,226.6"
12,2015,Agate Fossil Beds NM*,"13,264",$952.6


In [0]:
# Get 2016 dataset from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path_16 = "https://www.nps.gov/nature/customcf/NPS_Data_Visualization/docs/2016_VSE.pdf"
tabula.convert_into(pdf_path_16,"visitor_spent_2016.csv",output_format="csv",pages = [26,27,28,29,30,31,32,33,34,35,36,37,38,39],stream=True)

#Import the CSV created above into a dataframe
df_2016 = pd.read_csv('visitor_spent_2016.csv')

#create a new dataframe for the National Park Spending in 2016
spending_2016 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
spending_2016['park_unit'] = df_2016.iloc[:,0][5:]
spending_2016['recreation_visits'] = df_2016.iloc[:,1][5:]
spending_2016['visitor_spending_$000s']=df_2016.iloc[:,2][5:]
spending_2016['year'] = 2016

spending_2016.head()

,year,park_unit,recreation_visits,visitor_spending_$000s
5,2016,Abraham Lincoln Birthplace NHP,"252,495","$14,763.1"
6,2016,Acadia NP,"3,303,394","$274,208.8"
7,2016,Adams NHP,"199,300","$11,652.8"
8,2016,African Burial Ground NM,"46,526","$2,720.4"
9,2016,Agate Fossil Beds NM*,"15,555","$1,140.4"


In [0]:
# Get 2017 dataset from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path_17 = "https://www.nps.gov/nature/customcf/NPS_Data_Visualization/docs/NPS_2017_Visitor_Spending_Effects.pdf"
tabula.convert_into(pdf_path_17,"visitor_spent_2017.csv",output_format="csv",pages = [25,26,27,28,29,30,31,32,33,34,35,36,37],stream=True)

#Import the CSV created above into a dataframe
df_2017 = pd.read_csv('visitor_spent_2017.csv',error_bad_lines=False)

#create a new dataframe for the National Park Spending in 2017
spending_2017 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
spending_2017['park_unit'] = df_2017.iloc[:,0][7:]
spending_2017['recreation_visits'] = df_2017.iloc[:,1][7:]
spending_2017['visitor_spending_$000s']=df_2017.iloc[:,2][7:]
spending_2017['year'] = 2017

spending_2017.head()

b'Skipping line 38: expected 6 fields, saw 7\nSkipping line 39: expected 6 fields, saw 7\nSkipping line 40: expected 6 fields, saw 7\nSkipping line 41: expected 6 fields, saw 7\nSkipping line 42: expected 6 fields, saw 7\nSkipping line 43: expected 6 fields, saw 7\nSkipping line 44: expected 6 fields, saw 7\nSkipping line 45: expected 6 fields, saw 7\nSkipping line 46: expected 6 fields, saw 7\nSkipping line 47: expected 6 fields, saw 7\nSkipping line 48: expected 6 fields, saw 7\nSkipping line 49: expected 6 fields, saw 7\nSkipping line 50: expected 6 fields, saw 7\nSkipping line 51: expected 6 fields, saw 7\nSkipping line 52: expected 6 fields, saw 7\nSkipping line 53: expected 6 fields, saw 7\nSkipping line 54: expected 6 fields, saw 7\nSkipping line 55: expected 6 fields, saw 7\nSkipping line 56: expected 6 fields, saw 7\nSkipping line 57: expected 6 fields, saw 7\nSkipping line 58: expected 6 fields, saw 7\nSkipping line 59: expected 6 fields, saw 7\nSkipping line 60: expected 6 f

,year,park_unit,recreation_visits,visitor_spending_$000s
7,2017,Abraham Lincoln Birthplace NHP,"269,580","$15,412.8 242"
8,2017,Acadia NP,"3,509,271","$284,459.8 4,163"
9,2017,Adams NHP,"255,563","$14,611.5 200"
10,2017,African Burial Ground NM,"43,183","$2,469.0 31"
11,2017,Agate Fossil Beds NM*,"32,039","$2,299.0 35"


In [0]:
# Get 2018 dataset from the web
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
pdf_path_18 = "https://www.nps.gov/nature/customcf/NPS_Data_Visualization/docs/NPS_2018_Visitor_Spending_Effects.pdf"
tabula.convert_into(pdf_path_18,"visitor_spent_2018.csv",output_format="csv",pages = [26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44],stream=True)

#Import the CSV created above into a dataframe
df_2018 = pd.read_csv('visitor_spent_2018.csv',error_bad_lines=False)

#create a new dataframe for the National Park Spending in 2018
spending_2018 = pd.DataFrame(columns = ['year','park_unit', 'recreation_visits', 'visitor_spending_$000s'])

#import the necessary data from the csv dataframe - with updated column names
spending_2018['park_unit'] = df_2018.iloc[:,0][2:]
spending_2018['recreation_visits'] = df_2018.iloc[:,1][2:]
spending_2018['visitor_spending_$000s']=df_2018.iloc[:,2][2:]
spending_2018['year'] = 2018

spending_2018.head()

b'Skipping line 23: expected 6 fields, saw 7\nSkipping line 24: expected 6 fields, saw 7\nSkipping line 25: expected 6 fields, saw 7\nSkipping line 26: expected 6 fields, saw 7\nSkipping line 27: expected 6 fields, saw 7\nSkipping line 28: expected 6 fields, saw 7\nSkipping line 29: expected 6 fields, saw 7\nSkipping line 30: expected 6 fields, saw 7\nSkipping line 31: expected 6 fields, saw 7\nSkipping line 32: expected 6 fields, saw 7\nSkipping line 33: expected 6 fields, saw 7\nSkipping line 34: expected 6 fields, saw 7\nSkipping line 35: expected 6 fields, saw 7\nSkipping line 36: expected 6 fields, saw 7\nSkipping line 37: expected 6 fields, saw 7\nSkipping line 38: expected 6 fields, saw 7\nSkipping line 39: expected 6 fields, saw 7\nSkipping line 40: expected 6 fields, saw 7\nSkipping line 41: expected 6 fields, saw 7\nSkipping line 42: expected 6 fields, saw 7\nSkipping line 43: expected 6 fields, saw 7\nSkipping line 44: expected 6 fields, saw 7\nSkipping line 45: expected 6 f

,year,park_unit,recreation_visits,visitor_spending_$000s
2,2018,Abraham Lincoln Birthplace NHP,"260,964","$15,411 229"
3,2018,Acadia NP * !,"3,537,575","$387,654 5,602"
4,2018,Adams NHP,"121,007","$7,146 94"
5,2018,African Burial Ground NM,"45,036","$2,660 31"
6,2018,Agate Fossil Beds NM *,"16,238","$1,206 17"


In [0]:
# Union 2012-2018 dataframes into one big dataframe
spending_all = pd.concat([spending_2012,spending_2013,spending_2014,spending_2015,spending_2016,spending_2017,spending_2018],ignore_index=True)
spending_all.groupby("year").count()

,park_unit,recreation_visits,visitor_spending_$000s
year,,,
2012,382,395,395
2013,390,397,406
2014,390,397,406
2015,399,417,432
2016,398,417,443
2017,210,214,226
2018,121,119,124


In [0]:
# Since the table contains the data for National Monuments, National Historial Parks, we filtered to include National Parks only
spending = spending_all[spending_all["park_unit"].str.contains("NP",na=False)]

# Since some National Park Reserved only-NPRES-meet the above criteria, we need to exclude them as well
spending_all_NP = spending[~spending["park_unit"].str.contains("NPRES")]

# Sanity check to make sure only NP data are included
spending_all_NP.groupby("park_unit").size()

park_unit
Acadia NP          6
Acadia NP * !      1
Arches NP *        1
Arches NP*         6
Badlands NP        6
                  ..
Yellowstone NP*    6
Yosemite NP * !    1
Yosemite NP*       6
Zion NP * !        1
Zion NP*           6
Length: 69, dtype: int64

#Write Vistor Spending Data to .csv File

In [0]:
# Convert dataframe to csv
spending_all_NP.to_csv("vistors_spending_all_NPs.csv", sep=',',index=False)
# reading csv file  
pd.read_csv("vistors_spending_all_NPs.csv")


,year,park_unit,recreation_visits,visitor_spending_$000s
0,2012,Acadia NP,"2,431,052","$200,920.0"
1,2012,Arches NP*,"1,070,577","$115,859.9"
2,2012,Badlands NP,"883,406","$52,145.4"
3,2012,Big Bend NP,"292,055","$24,230.1"
4,2012,Biscayne NP,"495,613","$29,193.6"
...,...,...,...,...
331,2018,Wind Cave NP *,"656,398","$66,631 1,009"
332,2018,Wrangell - St Elias NP&PRES,"79,450","$120,679 1,486"
333,2018,Yellowstone NP *,"4,115,001","$512,566 7,089"
334,2018,Yosemite NP * !,"4,009,438","$495,245 6,184"


#Tabula data extraction for Budget Allocation on each Nation Park

In [0]:
#change the path to a new budget folder:

os.chdir('/content/drive/My Drive/budget_files')

# 2018 budget data
#Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_18 = "https://www.nps.gov/aboutus/upload/FY-2018-NPS-Greenbook.pdf"
tabula.convert_into(budget_path_18,"budget_2018.csv",output_format="csv",pages = [152,153,154,155,156,157,158,159,160,161,162,163,164],stream=True)

#Import the CSV created above into a dataframe
bg_2018 = pd.read_csv('budget_2018.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2018 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2018['park_unit'] = bg_2018.iloc[:,0][5:]
budget_2018['budget_$000s']=bg_2018.iloc[:,-1][5:]
budget_2018['year'] = 2018

#remove all unnecessary characters from the Park Unit name
budget_2018['park_unit'] = budget_2018['park_unit'].str.replace("[0-9]",'')
budget_2018['park_unit'] = budget_2018['park_unit'].str.replace(",",'')

budget_2018.head()



,year,park_unit,budget_$000s
5,2018,Abraham Lincoln Birthplace NHS,"1,274"
6,2018,Acadia NP,"7,684"
7,2018,Adams NHP,"2,364"
8,2018,African Burial Grounds NM,"1,857"
9,2018,Agate Fossil Beds NM,874


In [0]:
# 2017 budget data
#Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_17 = "https://www.nps.gov/aboutus/upload/FY17-NPS-Greenbook-for-website.pdf"
tabula.convert_into(budget_path_17,"budget_2017.csv",output_format="csv",pages = [170,171,172,173,174,175,176,177,178,179,180],stream=True)

#Import the CSV created above into a dataframe
bg_2017 = pd.read_csv('budget_2017.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2017 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2017['park_unit'] = bg_2017.iloc[:,0][5:]
budget_2017['budget_$000s']=bg_2017.iloc[:,-1][5:]
budget_2017['year'] = 2017

#remove all unnecessary characters from the Park Unit name
budget_2017['park_unit'] = budget_2017['park_unit'].str.replace("[0-9]",'')
budget_2017['park_unit'] = budget_2017['park_unit'].str.replace(",",'')

budget_2017.head()

b'Skipping line 38: expected 5 fields, saw 8\nSkipping line 39: expected 5 fields, saw 8\nSkipping line 40: expected 5 fields, saw 8\nSkipping line 41: expected 5 fields, saw 8\nSkipping line 42: expected 5 fields, saw 8\nSkipping line 43: expected 5 fields, saw 8\nSkipping line 44: expected 5 fields, saw 8\nSkipping line 45: expected 5 fields, saw 8\nSkipping line 46: expected 5 fields, saw 8\nSkipping line 47: expected 5 fields, saw 8\nSkipping line 48: expected 5 fields, saw 8\nSkipping line 49: expected 5 fields, saw 8\nSkipping line 50: expected 5 fields, saw 8\nSkipping line 51: expected 5 fields, saw 8\nSkipping line 52: expected 5 fields, saw 8\nSkipping line 53: expected 5 fields, saw 8\nSkipping line 54: expected 5 fields, saw 8\nSkipping line 55: expected 5 fields, saw 8\nSkipping line 56: expected 5 fields, saw 8\nSkipping line 57: expected 5 fields, saw 8\nSkipping line 58: expected 5 fields, saw 8\nSkipping line 59: expected 5 fields, saw 8\nSkipping line 60: expected 5 f

,year,park_unit,budget_$000s
5,2017,Acadia NP,"8,194"
6,2017,Adams NHP,"2,529"
7,2017,African Burial Grounds NM,"1,992"
8,2017,Agate Fossil Beds NM,943
9,2017,Allegheny Portage Railroad NHS,"2,115"


In [0]:
# 2016 budget data
#Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_16 = "https://www.nps.gov/aboutus/upload/FY-2016-Greenbook.pdf"
tabula.convert_into(budget_path_16,"budget_2016.csv",output_format="csv",pages = [176,177,178,179,180,181,182,183,184,185,186,187],stream=True)

#Import the CSV created above into a dataframe
bg_2016 = pd.read_csv('budget_2016.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2016 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2016['park_unit'] = bg_2016.iloc[:,0][5:]
budget_2016['budget_$000s']=bg_2016.iloc[:,-1][5:]
budget_2016['year'] = 2016

#remove all unnecessary characters from the Park Unit name
budget_2016['park_unit'] = budget_2016['park_unit'].str.replace("[0-9]",'')
budget_2016['park_unit'] = budget_2016['park_unit'].str.replace(",",'')

budget_2016.head()

b'Skipping line 35: expected 6 fields, saw 8\nSkipping line 36: expected 6 fields, saw 8\nSkipping line 37: expected 6 fields, saw 8\nSkipping line 38: expected 6 fields, saw 8\nSkipping line 39: expected 6 fields, saw 8\nSkipping line 40: expected 6 fields, saw 8\nSkipping line 41: expected 6 fields, saw 8\nSkipping line 42: expected 6 fields, saw 8\nSkipping line 43: expected 6 fields, saw 8\nSkipping line 44: expected 6 fields, saw 8\nSkipping line 45: expected 6 fields, saw 8\nSkipping line 46: expected 6 fields, saw 8\nSkipping line 47: expected 6 fields, saw 8\nSkipping line 48: expected 6 fields, saw 8\nSkipping line 49: expected 6 fields, saw 8\nSkipping line 50: expected 6 fields, saw 8\nSkipping line 51: expected 6 fields, saw 8\nSkipping line 52: expected 6 fields, saw 8\nSkipping line 53: expected 6 fields, saw 8\nSkipping line 54: expected 6 fields, saw 8\nSkipping line 55: expected 6 fields, saw 8\nSkipping line 56: expected 6 fields, saw 8\nSkipping line 57: expected 6 f

,year,park_unit,budget_$000s
5,2016,Abraham Lincoln Birthplace NHS,"1,347"
6,2016,Acadia NP,"8,130"
7,2016,Adams NHP,"2,507"
8,2016,African Burial Grounds NM,"1,988"
9,2016,Agate Fossil Beds NM,930


In [0]:
# 2015 budget data
#Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_15 = "https://www.nps.gov/aboutus/upload/FY-2015-Greenbook-Linked.pdf"
tabula.convert_into(budget_path_15,"budget_2015.csv",output_format="csv",pages = [177,178,179,180,181,182,183,184,185,186,187],stream=True)

#Import the CSV created above into a dataframe
bg_2015 = pd.read_csv('budget_2015.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2015 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2015['park_unit'] = bg_2015.iloc[:,0][5:]
budget_2015['budget_$000s']=bg_2015.iloc[:,-1][5:]
budget_2015['year'] = 2015

#remove all unnecessary characters from the Park Unit name
budget_2015['park_unit'] = budget_2015['park_unit'].str.replace("[0-9]",'')
budget_2015['park_unit'] = budget_2015['park_unit'].str.replace(",",'')

budget_2015.head()

b'Skipping line 38: expected 5 fields, saw 8\nSkipping line 39: expected 5 fields, saw 8\nSkipping line 40: expected 5 fields, saw 8\nSkipping line 41: expected 5 fields, saw 8\nSkipping line 42: expected 5 fields, saw 8\nSkipping line 43: expected 5 fields, saw 8\nSkipping line 44: expected 5 fields, saw 8\nSkipping line 45: expected 5 fields, saw 8\nSkipping line 46: expected 5 fields, saw 8\nSkipping line 47: expected 5 fields, saw 8\nSkipping line 48: expected 5 fields, saw 8\nSkipping line 49: expected 5 fields, saw 8\nSkipping line 50: expected 5 fields, saw 8\nSkipping line 51: expected 5 fields, saw 8\nSkipping line 52: expected 5 fields, saw 8\nSkipping line 53: expected 5 fields, saw 8\nSkipping line 54: expected 5 fields, saw 8\nSkipping line 55: expected 5 fields, saw 8\nSkipping line 56: expected 5 fields, saw 8\nSkipping line 57: expected 5 fields, saw 8\nSkipping line 58: expected 5 fields, saw 8\nSkipping line 59: expected 5 fields, saw 8\nSkipping line 60: expected 5 f

,year,park_unit,budget_$000s
5,2015,A braham Lincoln Birth p lace NHS,"1,316"
6,2015,A cadia NP,"8,038"
7,2015,A d ams NHP,"2,423"
8,2015,A frican Burial Gro unds NM,"1,949"
9,2015,A g ate Fos s il Beds NM,920


In [0]:
# 2014 budget data
#Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_14 = "https://www.nps.gov/aboutus/upload/FY_2014_greenbook.pdf"
tabula.convert_into(budget_path_14,"budget_2014.csv",output_format="csv",pages = [184,185,186,187,188,189,190,191,192,193],stream=True)

#Import the CSV created above into a dataframe
bg_2014 = pd.read_csv('budget_2014.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2014 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2014['park_unit'] = bg_2014.iloc[:,0][5:]
budget_2014['budget_$000s']=bg_2014.iloc[:,-1][5:]
budget_2014['year'] = 2014

#remove all unnecessary characters from the Park Unit name
budget_2014['park_unit'] = budget_2014['park_unit'].str.replace("[0-9]",'')
budget_2014['park_unit'] = budget_2014['park_unit'].str.replace(",",'')

budget_2014.head()

,year,park_unit,budget_$000s
5,2014,Acadia NP,"7,920"
6,2014,Adams NHP,"2,451"
7,2014,African Burial Grounds NM,"1,952"
8,2014,Agate Fossil Beds NM,922
9,2014,Allegheny Portage Railroad NHS,"2,081"


In [0]:
# 2013 budget data
#Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_13 = "https://www.nps.gov/aboutus/upload/FY13_NPS_Greenbook.pdf"
tabula.convert_into(budget_path_13,"budget_2013.csv",output_format="csv",pages = [161,162,163,164,165,166,167,168,169,170],stream=True)

#Import the CSV created above into a dataframe
bg_2013 = pd.read_csv('budget_2013.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2013 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2013['park_unit'] = bg_2013.iloc[:,0][5:]
budget_2013['budget_$000s']=bg_2013.iloc[:,-1][5:]
budget_2013['year'] = 2013

#remove all unnecessary characters from the Park Unit name
budget_2013['park_unit'] = budget_2013['park_unit'].str.replace("[0-9]",'')
budget_2013['park_unit'] = budget_2013['park_unit'].str.replace(",",'')

budget_2013.head()

b'Skipping line 40: expected 6 fields, saw 8\nSkipping line 41: expected 6 fields, saw 8\nSkipping line 42: expected 6 fields, saw 8\nSkipping line 43: expected 6 fields, saw 8\nSkipping line 44: expected 6 fields, saw 8\nSkipping line 45: expected 6 fields, saw 8\nSkipping line 46: expected 6 fields, saw 8\nSkipping line 47: expected 6 fields, saw 8\nSkipping line 48: expected 6 fields, saw 8\nSkipping line 49: expected 6 fields, saw 8\nSkipping line 50: expected 6 fields, saw 8\nSkipping line 51: expected 6 fields, saw 8\nSkipping line 52: expected 6 fields, saw 8\nSkipping line 53: expected 6 fields, saw 8\nSkipping line 54: expected 6 fields, saw 8\nSkipping line 55: expected 6 fields, saw 8\nSkipping line 56: expected 6 fields, saw 8\nSkipping line 57: expected 6 fields, saw 8\nSkipping line 58: expected 6 fields, saw 8\nSkipping line 59: expected 6 fields, saw 8\nSkipping line 60: expected 6 fields, saw 8\nSkipping line 61: expected 6 fields, saw 8\nSkipping line 62: expected 6 f

,year,park_unit,budget_$000s
5,2013,Abraham Lincoln Birthplace NHS,"1,328"
6,2013,Acadia NP,"7,937"
7,2013,Adams NHP,"2,471"
8,2013,African Burial Grounds NM,"1,976"
9,2013,Agate Fossil Beds NM,930


In [0]:
# 2012 budget data
# Convert pdf table into tabula format - note: pages list contains the pages where the table belongs to in the report
budget_path_12 = "https://www.nps.gov/aboutus/upload/FY_2012_greenbook.pdf"
tabula.convert_into(budget_path_12,"budget_2012.csv",output_format="csv",pages = [247,248,249,250,251,252,253,254,255,256,257,258],stream=True)

#Import the CSV created above into a dataframe
bg_2012 = pd.read_csv('budget_2012.csv',error_bad_lines=False)

#create a new dataframe for the National Park Budget in 2018
budget_2012 = pd.DataFrame(columns = ['year','park_unit','budget_$000s'])

#import the necessary data from the csv dataframe - with updated column names
budget_2012['park_unit'] = bg_2012.iloc[:,0][5:]
budget_2012['budget_$000s']=bg_2012.iloc[:,-1][5:]
budget_2012['year'] = 2012

#remove all unnecessary characters from the Park Unit name
budget_2012['park_unit'] = budget_2012['park_unit'].str.replace("[0-9]",'')
budget_2012['park_unit'] = budget_2012['park_unit'].str.replace(",",'')

budget_2012.head()

b'Skipping line 40: expected 6 fields, saw 9\nSkipping line 41: expected 6 fields, saw 9\nSkipping line 42: expected 6 fields, saw 9\nSkipping line 43: expected 6 fields, saw 9\nSkipping line 44: expected 6 fields, saw 9\nSkipping line 45: expected 6 fields, saw 9\nSkipping line 46: expected 6 fields, saw 9\nSkipping line 47: expected 6 fields, saw 9\nSkipping line 48: expected 6 fields, saw 9\nSkipping line 49: expected 6 fields, saw 9\nSkipping line 50: expected 6 fields, saw 9\nSkipping line 51: expected 6 fields, saw 9\nSkipping line 52: expected 6 fields, saw 9\nSkipping line 53: expected 6 fields, saw 9\nSkipping line 54: expected 6 fields, saw 9\nSkipping line 55: expected 6 fields, saw 9\nSkipping line 56: expected 6 fields, saw 9\nSkipping line 57: expected 6 fields, saw 9\nSkipping line 58: expected 6 fields, saw 9\nSkipping line 59: expected 6 fields, saw 9\nSkipping line 60: expected 6 fields, saw 9\nSkipping line 61: expected 6 fields, saw 9\nSkipping line 62: expected 6 f

,year,park_unit,budget_$000s
5,2012,National Park Service Park Units,NaN
6,2012,Abraham Lincoln Birthplace NHS,"1,338"
7,2012,Acadia NP,"8,017"
8,2012,Adams NHP,"3,370"
9,2012,African Burial Grounds NM,"1,991"


In [0]:
# Union all dataframes into one big dataframe
budget_all = pd.concat([budget_2012,budget_2013,budget_2014,budget_2015,budget_2016,budget_2017,budget_2018],ignore_index=True)
#budget_all.groupby("year").count()

# Filter dataframe to include National Parks only

budgets = budget_all[budget_all["park_unit"].str.contains("NP",na=False)]

# Since some National Park Reserved only-NPRES-meet the above criteria, we need to exclude them as well
budget_all_NP = budgets[~budgets["park_unit"].str.contains("NPRES")]

# Sanity check to make sure only NP data are included
#budget_all_NP.groupby("park_unit").size()
budgets.sort_values(by=['year'])

,year,park_unit,budget_$000s
2,2012,Acadia NP,"8,017"
14,2012,Arches NP,"1,943"
19,2012,Badlands NP,"4,397"
23,2012,Big Bend NP,"7,242"
24,2012,Big Cypress NPres,"7,292"
27,2012,Big Thicket NPres,"3,031"
29,2012,Biscayne NP,"4,556"
30,2012,Black Canyon of the Gunnison NP,"1,669"
61,2013,Biscayne NP,"4,297"
59,2013,Big Thicket NPres,"2,608"
